# Spectrum

In [1]:
import pandas as pd
import astropy.units as u
import numpy as np
import matplotlib.pyplot as plt
from astroquery.simbad import Simbad
from astropy.coordinates import SkyCoord
from pandas.plotting import scatter_matrix

In [2]:
catalog = pd.read_csv("markeddata/gevtevsimbad_TeVspectrum.txt", sep='\t', index_col=0)

In [3]:
for i in catalog.columns:
    print(i)

level_0
gev_0FGL_Name
gev_1FGL_Name
gev_1FHL_Name
gev_2FGL_Name
gev_ASSOC1
gev_ASSOC2
gev_ASSOC_GAM1
gev_ASSOC_GAM2
gev_ASSOC_GAM3
gev_ASSOC_TEV
gev_CLASS1
gev_Conf_68_PosAng
gev_Conf_68_SemiMajor
gev_Conf_68_SemiMinor
gev_Conf_95_PosAng
gev_Conf_95_SemiMajor
gev_Conf_95_SemiMinor
gev_Cutoff
gev_DEJ2000
gev_Energy_Flux100
gev_Exp_Index
gev_Extended_Source_Name
gev_Flags
gev_Flux1000
gev_Flux10000_100000
gev_Flux1000_3000
gev_Flux100_300
gev_Flux3000_10000
gev_Flux300_1000
gev_Flux30_100
gev_Flux_Density
gev_Flux_History
gev_Flux_Peak
gev_GLAT
gev_GLON
gev_Peak_Interval
gev_Pivot_Energy
gev_PowerLaw_Index
gev_RAJ2000
gev_ROI_num
gev_Signif_Avg
gev_Signif_Curve
gev_Signif_Peak
gev_Source_Name
gev_Spectral_Index
gev_SpectrumType
gev_Sqrt_TS10000_100000
gev_Sqrt_TS1000_3000
gev_Sqrt_TS100_300
gev_Sqrt_TS3000_10000
gev_Sqrt_TS300_1000
gev_Sqrt_TS30_100
gev_TEVCAT_FLAG
gev_Time_Peak
gev_Unc_Cutoff
gev_Unc_Energy_Flux100
gev_Unc_Exp_Index
gev_Unc_Flux1000
gev_Unc_Flux10000_100000
gev_Unc_Flux

In [4]:
catalog.head()

,level_0,gev_0FGL_Name,gev_1FGL_Name,gev_1FHL_Name,gev_2FGL_Name,gev_ASSOC1,gev_ASSOC2,gev_ASSOC_GAM1,gev_ASSOC_GAM2,gev_ASSOC_GAM3,...,tev_tevcat_name,tev_tgevcat_id,tev_tgevcat_name,tev_where,marked,tev_0.3TeV,tev_1TeV,tev_3TeV,tev_10TeV,tev_30TeV
0,79,b' ',b' ',b' ',b' ',b' ',b' ',b' ',b' ',b' ',...,b'TeV J1745-290d',107,b'TeV J1745-2900',b'gal',0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1,81,b' ',b'1FGL J1746.4-2849c',b'1FHL J1746.3-2851 ',b'2FGL J1746.6-2851c',b'PWN G0.13-0.11 ',b' ',b' ',b'3EG J1746-2851',b' ',...,b'TeV J1746-289',-9223372036854775808,b'',b'gal',0,1.647138e-11,8.320850e-14,2.778103e-14,1.856180e-15,3.184437e-16
2,108,b' ',b'1FGL J1746.4-2849c',b'1FHL J1746.3-2851 ',b'2FGL J1746.6-2851c',b'PWN G0.13-0.11 ',b' ',b' ',b'3EG J1746-2851',b' ',...,b'TeV J1746-289',-9223372036854775808,b'',b'gal',0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
3,97,b'0FGL J2032.2+4122',b'1FGL J2032.2+4127 ',b'1FHL J2032.1+4125 ',b'2FGL J2032.2+4126 ',b'LAT PSR J2032+4127 ',b' ',b'1AGL J2032+4102',b' ',b' ',...,b'TeV J2032+415',146,b'TeV J2031+4133',b'gal',1,2.151681e-11,9.163997e-13,1.292041e-13,9.049150e-15,5.995017e-16
4,37,b'0FGL J1018.2-5858',b'1FGL J1018.6-5856 ',b'1FHL J1018.9-5855 ',b'2FGL J1019.0-5856 ',b'1FGL J1018.6-5856 ',b' ',b' ',b' ',b' ',...,b'TeV J1018-589',45,b'TeV J1018-5856',b'gal',1,5.020464e-12,2.787421e-13,2.419200e-14,2.896730e-15,6.433307e-16


Columns contained spectrum in gev:

In [5]:
gev_spectrum_columns = [
    'gev_Flux1000',
    'gev_Flux10000_100000',
    'gev_Flux1000_3000',
    'gev_Flux100_300',
    'gev_Flux3000_10000',
    'gev_Flux300_1000',
    'gev_Flux30_100',
]

Columns contained spectrum in simbad:

In [6]:
s_spectrum_columns = [
    's_FLUX_U', 
    's_FLUX_B', 
    's_FLUX_V', 
    's_FLUX_R', 
    's_FLUX_I',
    's_FLUX_G', 
    's_FLUX_J', 
    's_FLUX_H',
    's_FLUX_K', 
    's_FLUX_u',
    's_FLUX_g', 
    's_FLUX_r',
    's_FLUX_i',    
    's_FLUX_z',
    ]

Columns contained spectrum in tev:

In [7]:
tev_spectrum_columns =  [
    'tev_0.3TeV', 
    'tev_1TeV', 
    'tev_3TeV',  
    'tev_10TeV',  
    'tev_30TeV' 
]

In [8]:
spectrum_columns = gev_spectrum_columns + s_spectrum_columns + tev_spectrum_columns

In [9]:
gevoptics_spectrum = catalog[spectrum_columns + ['s_MAIN_ID'] + ['marked']].copy()

In [10]:
for j in s_spectrum_columns:
    gevoptics_spectrum[j] = 10**(-gevoptics_spectrum[j])

In [11]:
gevoptics_spectrum = gevoptics_spectrum.fillna(0)

In [12]:
display(gevoptics_spectrum)

,gev_Flux1000,gev_Flux10000_100000,gev_Flux1000_3000,gev_Flux100_300,gev_Flux3000_10000,gev_Flux300_1000,gev_Flux30_100,s_FLUX_U,s_FLUX_B,s_FLUX_V,...,s_FLUX_r,s_FLUX_i,s_FLUX_z,tev_0.3TeV,tev_1TeV,tev_3TeV,tev_10TeV,tev_30TeV,s_MAIN_ID,marked
0,3.874533e-08,3.055154e-10,3.184363e-08,1.613918e-08,5.155299e-09,9.678488e-08,0.0,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,b'GCIRS 8',0
1,2.752177e-08,4.572024e-10,2.235459e-08,1.851010e-09,4.686388e-09,4.536945e-08,0.0,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.0,1.647138e-11,8.320850e-14,2.778103e-14,1.856180e-15,3.184437e-16,b'[MCD2010] 13',0
2,2.752177e-08,4.572024e-10,2.235459e-08,1.851010e-09,4.686388e-09,4.536945e-08,0.0,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,b'[MCD2010] 13',0
3,2.458654e-08,3.071351e-10,1.885649e-08,5.618088e-08,5.075542e-09,4.394119e-08,0.0,4.073800e-13,9.772367e-13,5.011870e-12,...,0.000000e+00,0.000000e+00,0.0,2.151681e-11,9.163997e-13,1.292041e-13,9.049150e-15,5.995017e-16,b'Schulte 21',1
4,2.713704e-08,1.420659e-10,2.192043e-08,2.666705e-07,3.261212e-09,1.141154e-07,0.0,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.0,5.020464e-12,2.787421e-13,2.419200e-14,2.896730e-15,6.433307e-16,b'2FGL J1019.0-5856',1
5,1.184660e-08,1.087441e-09,8.559550e-09,5.170882e-09,3.236763e-09,1.076956e-08,0.0,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.0,6.310589e-11,1.207092e-11,8.365091e-13,4.356098e-14,2.865129e-15,b'[KRL2007b] 332',0
6,1.909024e-08,2.244687e-10,1.828320e-08,4.639998e-07,1.954995e-09,1.100878e-07,0.0,9.549916e-13,5.888443e-13,5.370312e-12,...,1.258927e-11,4.168693e-11,0.0,2.891631e-10,2.077088e-11,1.649455e-12,7.704891e-14,2.742394e-15,b'V* V479 Sct',1
7,4.606472e-08,5.130483e-10,3.871007e-08,5.552702e-07,6.510779e-09,1.962727e-07,0.0,5.370312e-12,2.454711e-12,1.778279e-11,...,0.000000e+00,0.000000e+00,0.0,3.033067e-11,5.807976e-13,5.926255e-14,1.818965e-15,2.823374e-16,b'LS I +61 303',1
8,9.343741e-10,5.395927e-11,9.321550e-10,7.792275e-09,1.366562e-10,2.611374e-09,0.0,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,b'CXOU J095550.1+694034',0
9,2.178238e-08,9.998126e-10,1.540335e-08,5.515597e-07,5.319086e-09,8.277984e-08,0.0,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,b'SSTGC 519103',0


Normilization:

In [13]:
gevoptics_spectrum[spectrum_columns] = gevoptics_spectrum[spectrum_columns].div(gevoptics_spectrum.sum(axis=1), axis=0)

The cell displays all information we have about spectrums now.

In [14]:
display(gevoptics_spectrum)

,gev_Flux1000,gev_Flux10000_100000,gev_Flux1000_3000,gev_Flux100_300,gev_Flux3000_10000,gev_Flux300_1000,gev_Flux30_100,s_FLUX_U,s_FLUX_B,s_FLUX_V,...,s_FLUX_r,s_FLUX_i,s_FLUX_z,tev_0.3TeV,tev_1TeV,tev_3TeV,tev_10TeV,tev_30TeV,s_MAIN_ID,marked
0,2.049950e-01,1.616431e-03,1.684793e-01,8.538969e-02,2.727582e-02,5.120725e-01,0.0,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,b'GCIRS 8',0
1,2.671169e-01,4.437450e-03,2.169660e-01,1.796527e-02,4.548447e-02,4.403404e-01,0.0,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.0,1.598655e-04,8.075932e-07,2.696332e-07,1.801545e-08,3.090705e-09,b'[MCD2010] 13',0
2,2.671599e-01,4.438164e-03,2.170009e-01,1.796816e-02,4.549180e-02,4.404113e-01,0.0,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,b'[MCD2010] 13',0
3,2.458654e-08,3.071350e-10,1.885649e-08,5.618087e-08,5.075541e-09,4.394119e-08,0.0,4.073799e-13,9.772366e-13,5.011869e-12,...,0.000000e+00,0.000000e+00,0.0,2.151681e-11,9.163995e-13,1.292041e-13,9.049149e-15,5.995016e-16,b'Schulte 21',1
4,2.713703e-08,1.420658e-10,2.192042e-08,2.666704e-07,3.261211e-09,1.141154e-07,0.0,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.0,5.020462e-12,2.787420e-13,2.419199e-14,2.896729e-15,6.433305e-16,b'2FGL J1019.0-5856',1
5,2.820232e-01,2.588789e-02,2.037707e-01,1.230993e-01,7.705518e-02,2.563828e-01,0.0,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.0,1.502314e-03,2.873632e-04,1.991414e-05,1.037023e-06,6.820796e-08,b'[KRL2007b] 332',0
6,1.909023e-08,2.244685e-10,1.828319e-08,4.639996e-07,1.954994e-09,1.100877e-07,0.0,9.549910e-13,5.888439e-13,5.370309e-12,...,1.258926e-11,4.168690e-11,0.0,2.891630e-10,2.077087e-11,1.649454e-12,7.704886e-14,2.742392e-15,b'V* V479 Sct',1
7,4.606468e-08,5.130479e-10,3.871003e-08,5.552697e-07,6.510773e-09,1.962725e-07,0.0,5.370308e-12,2.454709e-12,1.778278e-11,...,0.000000e+00,0.000000e+00,0.0,3.033064e-11,5.807971e-13,5.926250e-14,1.818964e-15,2.823372e-16,b'LS I +61 303',1
8,7.498512e-02,4.330324e-03,7.480703e-02,6.253434e-01,1.096689e-02,2.095673e-01,0.0,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,b'CXOU J095550.1+694034',0
9,3.213384e-02,1.474945e-03,2.272336e-02,8.136728e-01,7.846831e-03,1.221186e-01,0.0,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,b'SSTGC 519103',0
